# Assignment 2 - Question 4
The objective of this assignment is to get you familiarize with  the  problem  of  `Linear Regression`.

## Instructions
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells.
- No inbuilt functions to be used until specified


Name: Aum Alok Khatlawala <br>
Roll Number: 2020113008

## Background about the dataset

TLDR: You have 4 independent variables (`float`) for each molecule. You can use a linear combination of these 4 independent variables to predict the bandgap (dependent variable) of each molecule.

You can read more about the problem in [Li et al, Bandgap tuning strategy by cations and halide ions of lead halide perovskites learned from machine learning, RSC Adv., 2021,11, 15688-15694](https://doi.org/10.1039/D1RA03117A).

In [1]:
import csv
import random
import numpy as np

In [2]:
all_molecules = list()

with open('bg_data.txt', 'r') as infile:
    input_rows = csv.DictReader(infile)
    
    for row in input_rows:
        current_mol = ([float(row['Cs']), float(row['FA']), float(row['Cl']), float(row['Br'])], float(row['Bandgap']))
        all_molecules.append(current_mol)

random.shuffle(all_molecules)


num_train = int(len(all_molecules) * 0.8)

# each point in x_train has 4 values - 1 for each feature
x_train = [x[0] for x in all_molecules[:num_train]]
# each point in y_train has 1 value - the bandgap of the molecule
y_train = [x[1] for x in all_molecules[:num_train]]

x_test = [x[0] for x in all_molecules[num_train:]]
y_test = [x[1] for x in all_molecules[num_train:]]

### 4.1 Implement a Linear Regression model that minimizes the MSE **without using any libraries**. You may use NumPy to vectorize your code, but *do not use numpy.polyfit* or anything similar.

4.1.1 Explain how you plan to implement Linear Regression in 5-10 lines.

<ol>
    <li> Initialise the weights and bias to a zerro array and a zero respectively. </li>
    <li> In each iteration, we calculate predicted value of y using the dot product of weights and the value of each x and adding bias. </li>
    <li> Then, we calculate the gradient by checking the deviation from actual value of y and taking dot product with x and then normalising. </li>
    <li> Then, we decrease weights by a factor depending on the learning rate and the gradient. </li>
    <li> After training the model, we test it and check how good it is using measures such as RMSE. </li>
</ol>

4.1.2 Implement Linear Regression using `x_train` and `y_train` as the train dataset.

4.1.2.1 Choose the best learning rate and print the learning rate for which you achieved the best MSE.

In [ ]:
learning_rates = [0.00001, 0.0001, 0.001, 0.01]
num_iterations = 10000
MSEs = []
for rate in learning_rates:
    num_molecules = len(x_train)
    num_features = len(x_train[0])
    weights = np.zeros(num_features)
    bias = 0
    for num_iter in range(num_iterations):
        x_arr = np.array(x_train)
        y_pred = []
        for x_val in x_arr:
            predicted = np.dot(weights, x_val) + bias
            y_pred.append(predicted)
        y_pred_arr = np.array(y_pred)
        y_arr = np.array(y_train)
        gradient = np.dot(x_arr.T, (y_pred_arr - y_arr)) / y_arr.size
        weights = weights - (rate * gradient)
        bias = bias - (rate * np.sum(y_pred_arr - y_arr))
    y_test_pred = []
    x_test_arr = np.array(x_test)
    for x_val in x_test_arr:
        predicted = np.dot(weights, x_val) + bias
        y_test_pred.append(predicted)
    y_test_pred_arr = np.array(y_test_pred)
    y_test_arr = np.array(y_test)
    MSE = sum([(y_test_pred_arr[i] - y_test_arr[i])**2 for i in range(len(x_test))]) / len(x_test)
    MSEs.append(float(MSE))

# print(MSEs)
blr = learning_rates[MSEs.index(min(MSEs))]
print("Best learning rate: ", blr)
print("MSE of best learning rate: ", min(MSEs))

4.1.3 Make a [Parity Plot](https://en.wikipedia.org/wiki/Parity_plot) of your model's bandgap predictions on the test set with the actual values.

In [ ]:
# Get the predictions of x_test into `y_pred`

y_pred = []
x_test_arr = np.array(x_test)
for x_val in x_test_arr:
    predicted = np.dot(weights, x_val) + bias
    y_pred.append(predicted)
y_test_pred_arr = np.array(y_pred)

#

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,20))
# print(len(y_test), len(y_pred))

ax.scatter(y_test, y_pred)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),
    np.max([ax.get_xlim(), ax.get_ylim()]),
]
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
ax.set_aspect('equal')
ax.set_xlim(lims)
ax.set_ylim(lims)

ax.set_title('Parity Plot of Custom Linear Regression')
ax.set_xlabel('Ground truth bandgap values')
ax.set_ylabel('Predicted bandgap values')
plt.show()

### 4.2 Implement Ridge regression
4.2.1 Explain Ridge regression briefly in 1-2 lines.

In situations when the independent variables are highly correlated, ridge regression is a technique for estimating the coefficients of multiple regression models. The goal is to estimate the coefficients of a multiple regression model in such a way that the magnitude of the coefficients is as less as possible.

4.2.2 Implement Ridge regression and make a table of different RMSE scores you achieved with different values of alpha. What does the parameter `alpha` do? How does it affect the results here? Explain in 5-10 lines in total. (You can use scikit-learn from this cell onwards)

In [ ]:
# ! pip install sklearn

In [5]:
# you should not have imported sklearn before this point
import sklearn.linear_model as sl_linear_model
from tabulate import tabulate
import matplotlib.pyplot as plt

alphas = [1e-15, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
RMSEs = []
for alpha in alphas:
    model = sl_linear_model.Ridge(alpha = alpha)
    model.fit(x_train, y_train)
    y_test_pred = model.predict(x_test)
    RMSE_sum = 0
    for i in range(len(y_test)):
        RMSE_sum += (y_test_pred[i] - y_test[i])**2
    RMSE_sum = RMSE_sum / len(y_test)
    RMSE = np.sqrt(RMSE_sum)
    RMSEs.append(RMSE)

headers = ['Alphas', 'RMSEs']
table = zip(alphas, RMSEs)
print(tabulate(table, headers = headers, tablefmt = "fancy_grid"))
# plt.plot(alphas, RMSEs)
# implement Ridge regression and make a table where you explore the effect of different values of `alpha`

╒══════════╤═══════════╕
│   Alphas │     RMSEs │
╞══════════╪═══════════╡
│   1e-15  │ 0.0866715 │
├──────────┼───────────┤
│   1e-05  │ 0.0866709 │
├──────────┼───────────┤
│   0.0001 │ 0.0866656 │
├──────────┼───────────┤
│   0.001  │ 0.0866128 │
├──────────┼───────────┤
│   0.01   │ 0.0860946 │
├──────────┼───────────┤
│   0.1    │ 0.081828  │
├──────────┼───────────┤
│   1      │ 0.101785  │
├──────────┼───────────┤
│  10      │ 0.31362   │
├──────────┼───────────┤
│ 100      │ 0.457101  │
╘══════════╧═══════════╛


The parameter alpha in Ridge regression is a regularisation parameter. It controls the complexity of the model. The model is punished for having high weights more severely the higher the value of alpha. The model is less penalised for having high weights the lower the value of alpha. Cross-validation is used to choose the alpha value. The extra penalty is equal to the square of the coefficients' magnitude. The majority of feature coefficients are cancelled out as alpha increases. More feature coefficients are employed when alpha decreases. Lowest RMSE encountered at alpha = 0.1

### 4.3 Implement Lasso regression
4.3.1 Explain Lasso regression briefly in 1-2 lines.

The Lasso Regression analysis approach is used to do both variable selection and regularisation in order to enhance the predictability and interpretability of the generated statistical model. L1 regularisation technique is used by the Lasso Regression algorithm. When there are more features, it is considered since feature selection is automatic.

4.3.2 Implement Lasso regression and make a table of different RMSE scores you achieved with different values of alpha. What does the parameter `alpha` do? How does it affect the results here? Explain in 5-10 lines in total.

In [ ]:
# implement Lasso regression and make a table where you explore the effect of different values of `alpha`

In [6]:
alphas = [1e-15, 0.00001, 0.0001, 0.001, 0.01, 0.1, 10, 100]
RMSEs = []
for alpha in alphas:
    model = sl_linear_model.Lasso(alpha = alpha)
    model.fit(x_train, y_train)
    y_test_pred = model.predict(x_test)
    RMSE_sum = 0
    for i in range(len(y_test)):
        RMSE_sum += (y_test_pred[i] - y_test[i])**2
    RMSE_sum = RMSE_sum / len(y_test)
    RMSE = np.sqrt(RMSE_sum)
    RMSEs.append(RMSE)

headers = ['Alphas', 'RMSEs']
table = zip(alphas, RMSEs)
print(tabulate(table, headers = headers, tablefmt = "fancy_grid"))
# plt.plot(alphas, RMSEs)

╒══════════╤═══════════╕
│   Alphas │     RMSEs │
╞══════════╪═══════════╡
│   1e-15  │ 0.0866715 │
├──────────┼───────────┤
│   1e-05  │ 0.0866173 │
├──────────┼───────────┤
│   0.0001 │ 0.0861296 │
├──────────┼───────────┤
│   0.001  │ 0.0816123 │
├──────────┼───────────┤
│   0.01   │ 0.0871958 │
├──────────┼───────────┤
│   0.1    │ 0.492503  │
├──────────┼───────────┤
│  10      │ 0.492503  │
├──────────┼───────────┤
│ 100      │ 0.492503  │
╘══════════╧═══════════╛


The Lasso regression hyperparameter alpha regulates how much regularisation is to be performed to the model. The model is more regularly distributed the greater the value of alpha. The model is less regularised the lower the value of alpha. Cross-validation is used to choose the alpha value. The extra penalty is equal to the magnitude of the coefficients in absolute terms. Lowest RMSE encountered at alpha = 0.001.